<a href="https://colab.research.google.com/github/K-Jadeja/HuggingFace-LLMs/blob/main/Finetuning_BERT_for_Chatbot_intent_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip  install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/training/"

Mounted at /content/gdrive


In [ ]:
import os
os.listdir(root_dir)

['train.csv', '.ipynb_checkpoints']

In [ ]:
import pandas as pd

root_path = f"{root_dir}training/train.csv"
df = pd.read_csv(f"{root_dir}train.csv")
df.head()

,category,text
0,SearchDynamicNews,Which city hosted a major international confer...
1,SearchDynamicNews,What is the current status of the cryptocurren...
2,SearchDynamicNews,What is the current status of the global clima...
3,NormalChat,I'm currently reading a mystery novel.
4,NormalChat,Have you seen the latest episode of that TV show?


In [ ]:
df['encoded_text'] = df['category'].astype('category').cat.codes

df.head(10)

,category,text,encoded_text
0,SearchDynamicNews,Which city hosted a major international confer...,1
1,SearchDynamicNews,What is the current status of the cryptocurren...,1
2,SearchDynamicNews,What is the current status of the global clima...,1
3,NormalChat,I'm currently reading a mystery novel.,0
4,NormalChat,Have you seen the latest episode of that TV show?,0
5,SearchDynamicNews,What are the updates on international trade ag...,1
6,SearchDynamicNews,Who is the CEO of Twitter?,1
7,SearchDynamicNews,What is the current situation of the pandemic ...,1
8,NormalChat,Board game nights are so much fun!,0
9,SearchDynamicNews,What is the breakthrough in cancer research?,1


In [ ]:
# df = df.drop(['category'], axis=1)
df.rename(columns={'encoded_text': 'Sentiment', 'text':'Phrase'}, inplace=True)
df.head(10)

,category,Phrase,Sentiment
0,SearchDynamicNews,Which city hosted a major international confer...,1
1,SearchDynamicNews,What is the current status of the cryptocurren...,1
2,SearchDynamicNews,What is the current status of the global clima...,1
3,NormalChat,I'm currently reading a mystery novel.,0
4,NormalChat,Have you seen the latest episode of that TV show?,0
5,SearchDynamicNews,What are the updates on international trade ag...,1
6,SearchDynamicNews,Who is the CEO of Twitter?,1
7,SearchDynamicNews,What is the current situation of the pandemic ...,1
8,NormalChat,Board game nights are so much fun!,0
9,SearchDynamicNews,What is the breakthrough in cancer research?,1


In [ ]:
import numpy as np

seq_len = 512
num_samples = len(df)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

Xids.shape

(116, 512)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
for i, phrase in enumerate(df['Phrase']):
    tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='tf')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

In [ ]:
# Xids
Xmask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [ ]:
# first extract sentiment column
arr = df['Sentiment'].values

# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1

In [ ]:
import tensorflow as tf

In [ ]:
# create the dataset object
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
dataset = dataset.map(map_func)

In [ ]:
dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(2,), dtype=tf.float64, name=None))>

In [ ]:
# we will split into batches of 16
batch_size = 6

# shuffle and batch - dropping any remaining samples that don't cleanly
# fit into a batch of 16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

In [ ]:
# set split size (90% training data) and calculate training set size
split = 0.9
size = int((Xids.shape[0]/batch_size)*split)

# get training and validation sets
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [ ]:
del dataset

In [ ]:
# AutoModel for PyTorch, TFAutoModel for TensorFlow
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # access pooled activations with [1]

# convert bert embeddings into 5 output classes
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
y = tf.keras.layers.Dense(arr.max()+1, activation='softmax', name='outputs')(x)

In [ ]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# (optional) freeze bert layer
#model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [ ]:
optimizer = tf.keras.optimizers.legacy.Adam(lr=1e-5, decay=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

Epoch 1/3
17/17 [==============================] - 27s 862ms/step - loss: 0.5844 - accuracy: 0.6961 - val_loss: 0.3777 - val_accuracy: 0.9167
Epoch 2/3
17/17 [==============================] - 12s 700ms/step - loss: 0.2246 - accuracy: 0.9510 - val_loss: 0.0622 - val_accuracy: 1.0000
Epoch 3/3
17/17 [==============================] - 12s 710ms/step - loss: 0.0470 - accuracy: 1.0000 - val_loss: 0.0113 - val_accuracy: 1.0000


In [ ]:
model.save('sentiment_model')

In [ ]:
# we just load the model from which directory it was saved to (eg '/sentiment_model')
model = tf.keras.models.load_model('sentiment_model')

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

Now, we're creating a function to tokenize input texts for prediction

In [ ]:
# initialize tokenizer from transformers
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prep_data(text):
    # tokenize to get input IDs and attention mask tensors
    tokens = tokenizer.encode_plus(text, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    # tokenizer returns int32 tensors, we need to return float64, so we use tf.cast
    return {'input_ids': tf.cast(tokens['input_ids'], tf.float64),
            'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}

In [ ]:
test = prep_data('Who is the most richest person in the world?')

In [ ]:
probs = model.predict(test)
probs[0] #We find probabilities but they are unreadable

1/1 [==============================] - 0s 131ms/step


array([0.0085841, 0.9914159], dtype=float32)

In [ ]:
import numpy as np

In [ ]:
import time

def output_to_intent(softmax_output):
    switcher = {
        0: "NormalChat",
        1: "SearchDynamicNews",
    }
    return switcher.get(softmax_output, "nothing")

def main():
  for i in range(10):
    user_input = "Did julie make a cameo in that marvel movie?"
    test = prep_data(user_input)
    probs = model.predict(test)
    softmax_output = np.argmax(probs)
    if i == 1:
      print("Hello, I'm a chatbot")
      print(user_input)
      print(output_to_intent(softmax_output))

start_time = time.time()
main()
end_time = time.time()
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

1/1 [==============================] - 0s 105ms/step
Hello, I'm a chatbot
Did julie make a cameo in that marvel movie?
SearchDynamicNews
1/1 [==============================] - 0s 108ms/step
Elapsed time: 1.8725767135620117 seconds
